In [42]:
import pandas as pd
import gpxo
import os
import random
import warnings
import gpxpy 

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore')

In [43]:
# Caminho da pasta que terá iteração nos seus arquivos
folder_path = 'C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/running_inactivity_prediction/athletes_activities/athlete1' 

In [44]:
# Atividade aleatória
random_activity_file = random.choice(os.listdir(folder_path))

path_example = os.path.join(folder_path, random_activity_file)

### Conferindo se algum arquivo está com problema de leitura

In [45]:
'''
# Caminho da pasta que terá iteração nos seus arquivos
folder_path = 'C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete1'

# Contador de linhas para servir de index
count = 0

# Itera por cada arquivo na pasta
for filename in os.listdir(folder_path):
    # Junta o nome do arquivo com o caminho da pasta
    full_path = os.path.join(folder_path, filename)    

    try:

        # Lê o arquivo no formato específico com a biblioteca gpxo
        gpx = gpxo.Track(full_path)

    except:

        # Printa o index e o nome do arquivo que ocorreu algum erro na leitura
        print(count, f"An error ocurred on {filename}")
    
    # Acréscimo no contador para reiniciar o loop
    count +=1

# Arquivo 6500685512.gpx com problema.
# UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 366: character maps to <undefined>.
# Problema resolvido: A atividade tinha um emoji e o decoder não estava conseguindo lê-lo. Se ocorrer o erro novamente, tentar remover o emoji do arquivo gpx.
'''

'\n# Caminho da pasta que terá iteração nos seus arquivos\nfolder_path = \'C:/Users/USER/Desktop/EstudosDados/Projetos/Corrida/physical_inactivity_prediction/athletes_activities/athlete1\'\n\n# Contador de linhas para servir de index\ncount = 0\n\n# Itera por cada arquivo na pasta\nfor filename in os.listdir(folder_path):\n    # Junta o nome do arquivo com o caminho da pasta\n    full_path = os.path.join(folder_path, filename)    \n\n    try:\n\n        # Lê o arquivo no formato específico com a biblioteca gpxo\n        gpx = gpxo.Track(full_path)\n\n    except:\n\n        # Printa o index e o nome do arquivo que ocorreu algum erro na leitura\n        print(count, f"An error ocurred on {filename}")\n    \n    # Acréscimo no contador para reiniciar o loop\n    count +=1\n\n# Arquivo 6500685512.gpx com problema.\n# UnicodeDecodeError: \'charmap\' codec can\'t decode byte 0x90 in position 366: character maps to <undefined>.\n# Problema resolvido: A atividade tinha um emoji e o decoder não

#### Demonstração dos valores após leitura dos arquivos com diferentes bibliotecas

In [46]:
#gpxo.Track
gpxo_example = gpxo.Track(path_example).data.reset_index()
gpxo_example

,time,latitude (°),longitude (°),distance (km),compass (°),duration (s),velocity (km/h),elevation (m)
0,2021-06-30 08:26:24,-22.939939,-43.178387,0.000000,180.000000,0.0,3.5892,12.3
1,2021-06-30 08:26:25,-22.939948,-43.178387,0.000997,176.716535,1.0,3.4002,12.3
2,2021-06-30 08:26:26,-22.939956,-43.178386,0.001889,176.716947,2.0,3.2004,12.3
3,2021-06-30 08:26:27,-22.939964,-43.178386,0.002775,176.716947,3.0,3.2004,12.3
4,2021-06-30 08:26:28,-22.939972,-43.178385,0.003667,176.716947,4.0,3.2004,12.3
...,...,...,...,...,...,...,...,...
5040,2021-06-30 10:07:37,-22.939686,-43.178761,25.766875,0.000000,6073.0,0.0000,12.6
5041,2021-06-30 10:07:38,-22.939686,-43.178761,25.766875,0.000000,6074.0,0.0000,12.6
5042,2021-06-30 10:07:39,-22.939686,-43.178761,25.766875,0.000000,6075.0,0.0000,12.6
5043,2021-06-30 10:07:40,-22.939686,-43.178761,25.766875,0.000000,6076.0,0.0000,12.6


In [47]:
#gpxpy
gpx = open(path_example, 'r')

gpxpy_example = gpxpy.parse(gpx)
gpxpy_example

GPX(tracks=[GPXTrack(name='Pedalada matinal', segments=[GPXTrackSegment(points=[...])])])

#### Criando as features com os dados extraidos de cada arquivo

In [48]:
# Dicionário com as features que serão criadas com o valor de cada atividade
df_athlete_dict = {
    'activity_date': [],
    'activity_type': [],
    'total_distance (km)': [], 
    'total_time (min)': [],
    'pace (min/km)': [],
    'speed (km/h)': []    
}

# Itera por cada arquivo na pasta
for filename in os.listdir(folder_path):
    # Junta o nome do arquivo com o caminho da pasta
    full_path = os.path.join(folder_path, filename)

    # Lê o arquivo .gpx com as bibliotecas gpxo e gpxpy
    gpx1 = gpxo.Track(full_path).data.reset_index() 
    gpx2 = gpxpy.parse(open(full_path, 'r'))
        
    # Data da atividade (a partir da primeira linha do df original com os pontos marcados)
    df_athlete_dict['activity_date'].append(gpx1['time'][0].normalize())

    # Tipo da atividade
    df_athlete_dict['activity_type'].append(gpx2.tracks[0].type)

    # Distância total em km
    total_dist = gpx1['distance (km)'].max()
    df_athlete_dict['total_distance (km)'].append(round(total_dist,2))

    # Tempo total em minutos
    total_min = (gpx1['time'].max() - gpx1['time'].min()).total_seconds() // 60
    df_athlete_dict['total_time (min)'].append(total_min)

    # Pace (min/km)
    if total_dist == 0:
        df_athlete_dict['pace (min/km)'].append(0)
    else:
        df_athlete_dict['pace (min/km)'].append(round(total_min / total_dist, 2))

    # Velocidade em km/h
    df_athlete_dict['speed (km/h)'].append(round(total_dist / (total_min / 60),2))

#### Criando um Dataframe novo a partir do dicionário

In [50]:
df_athlete_activities = pd.DataFrame(df_athlete_dict)

#### Ordenando a data da menor pra maior e resetando index

In [51]:
df_athlete_activities.sort_values('activity_date', ascending=True, inplace=True)
df_athlete_activities.reset_index(drop=True, inplace= True)

In [52]:
df_athlete_activities

,activity_date,activity_type,total_distance (km),total_time (min),pace (min/km),speed (km/h)
0,2020-07-09,running,6.03,39.0,6.47,9.27
1,2020-07-10,cycling,25.35,140.0,5.52,10.87
2,2020-07-12,running,7.57,55.0,7.27,8.25
3,2020-07-16,running,3.61,23.0,6.38,9.40
4,2020-07-20,cycling,25.66,127.0,4.95,12.12
...,...,...,...,...,...,...
109,2022-06-02,running,6.25,41.0,6.56,9.15
110,2022-06-07,running,8.51,54.0,6.35,9.45
111,2022-06-09,running,8.12,55.0,6.77,8.86
112,2022-06-13,running,11.34,73.0,6.44,9.32
